In [1]:
#----------------- Notebook for Hyperdrive & AutoML run ---------------------

In [2]:
##1. Initialise Workspace

In [5]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-134233")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-134233
Azure region: southcentralus
Subscription id: 9a7511b8-150f-4a58-8528-3e7d50216c31
Resource group: aml-quickstarts-134233


In [6]:
##2. Create AmlCompute Cluster

In [7]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

from azureml.core.compute_target import ComputeTargetException

cpu_cluster_name = "cpu-cluster1"

try: #--- Verify that cluster does not exist already
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
    
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [8]:
##3. Create Project Directory
##4. Create an estimator using SKLEARN
##5. Create hyperdrive config

In [9]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
#--from azureml.core import ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling( {
        "max_iter": choice(range(10,31)) ,
        "--C": uniform(0,10),
    }
)

# Specify a Policy
policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=1, delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory="./",
                    compute_target=cpu_cluster,
                    entry_script='train.py'
                   )


# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est,
                  hyperparameter_sampling=ps, 
                  policy= policy,
                  primary_metric_name="Accuracy",
              primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                  max_total_runs=20,
                  max_concurrent_runs=4)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [10]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(hyperdrive_config)

In [11]:
import joblib
from azureml.core import Model
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)
assert(hyperdrive_run.get_status() == "Completed")

hyperdrive_best_run = hyperdrive_run.get_best_run_by_primary_metric()

model= hyperdrive_best_run.register_model(model_name='udacity-hyperdrive-project', model_path='./outputs/model.pkl',model_framework=Model.Framework.SCIKITLEARN,model_framework_version='0.19.1')


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_a511da17-5615-48f3-a3ce-8570a556fece
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_a511da17-5615-48f3-a3ce-8570a556fece?wsid=/subscriptions/9a7511b8-150f-4a58-8528-3e7d50216c31/resourcegroups/aml-quickstarts-134233/workspaces/quick-starts-ws-134233

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-01-10T15:46:03.195024][API][INFO]Experiment created<END>\n""<START>[2021-01-10T15:46:03.679885][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2021-01-10T15:46:03.852537][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2021-01-10T15:46:04.4243307Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>

Execution Summary
RunId: HD_a511da17-5615-48f3-a3ce-8570a556fece
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_a511da17-5615-48f3-a3ce-8570a556fece?wsid=/subscriptions/9a751

In [12]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
web_path = ['https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv']
ds = TabularDatasetFactory.from_delimited_files(path=web_path)

In [17]:
from train import clean_data
from sklearn.model_selection import train_test_split
import pandas as pd
from azureml.core import Workspace, Datastore

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

X_train, X_test, Y_train, Y_test = train_test_split(x,y,test_size=0.30)
df_train = pd.concat([X_train, Y_train],axis=1)

ds_train = TabularDatasetFactory.register_pandas_dataframe(dataframe=df_train, target=Datastore.get_default(ws),name='train_data')


Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/b60bc852-60e0-43f2-bd85-71a85756aaac/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.


In [18]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy",
    training_data= ds_train,
    compute_target= cpu_cluster,
    label_column_name= 'y',
    n_cross_validations=5)

In [19]:
# Submit your automl run

### YOUR CODE HERE ###
b_experiment = Experiment(ws, "automl_experiment")
automl_run = b_experiment.submit(config=automl_config, show_output=True)

Running on remote.
No run_configuration provided, running on cpu-cluster1 with default configuration
Running on remote compute: cpu-cluster1
Parent Run ID: AutoML_9993f4cc-5d31-4471-8477-f0ddf892ebd0

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+----------

In [21]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output=True)
assert(automl_run.get_status() == "Completed")

automl_best_run = automl_run.get_best_child()

model= automl_best_run.register_model(model_name='udacity-automl-project', model_path='./outputs/model.pkl',model_framework=Model.Framework.SCIKITLEARN,model_framework_version='0.19.1')


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…



****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+--------------------------------------+
|Size of the smallest class       |Name/Label of the smallest class |Number of samples in the training data|
+=================================+=================================+======================================+
|2557                             |1                                |23065                                 |
+---------------------------------+----------